# Building A Database For Crime Reports

In this notebook, we will use Postgres to build a database for storing data about crimes that occurred in Boston. The dataset is available in the file boston.csv.

We will create a database named crimes_db, with a schema named crimes, and a table named boston_crimes with data from the boston.csv file. We will also create readonly and readwrite groups with appropriate priviledges, as well as example users for each of these groups.

## Creating the Crime Database

We will start by creating the database for storing our crime data, as well as the schema.

In [1]:
import psycopg2
import csv

In [ ]:
# crime_db does not exist yet, so we connect to postgres db first and create the crime_db from there.
conn = psycopg2.connect(dbname="postgres", user="postgres")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("CREATE DATABASE crime_db;")
conn.close()

In [12]:
# Connect to crime_db and create schema named crimes.
conn = psycopg2.connect(dbname="crime_db", user="postgres")
cursor = conn.cursor()
cursor.execute("CREATE SCHEMA crimes;")

## Obtaining the Column Names and Sample

Before we load data into our table, let's first understand the crime dataset so we can choose the right datatypes to use in our table.

In [2]:
with open("boston.csv", "r") as file:
    row_list = list(csv.reader(file))
    
col_headers = row_list[0]
first_row = row_list[1]

print("Column Headers: ", col_headers)
print("First Data Row: ", first_row)

Column Headers:  ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
First Data Row:  ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

Let's write a function -- get_col_set() -- that will help us identify proper datatypes for the columns. This function will return a set of all distinct values in a column of a CSV file. This will allow us to see if any columns can be of enumerated type and will allow us to easily calculate the maximum length of data in column so we can appropriately set VARCHAR.

We will start off by using this function to find the number of different values in each column of the boston.csv file.

In [3]:
def get_col_set(csv_filename, col_index):
    unique_values = set()
    
    with open(csv_filename, 'r') as file:
        file_list = list(csv.reader(file))
        for item in file_list[1:]:
            unique_values.add(item[col_index])
    
    return unique_values

In [4]:
num_distinct_values = []
for index in range(0, len(col_headers)):
    num_distinct_values.append(len(get_col_set("boston.csv", index)))
    
print(num_distinct_values)

[298329, 219, 239, 1177, 7, 18177, 18177]


Columns with a low number of distinct values are usually good candidates for enumerated data type. (In this case, column 5 has 7 distinct values, and corresponds to day_of_the_week, which we can definitely use enumerated data type for).

In addition, for our text columns, we want to know what the max-length value is so we can set the appropriate length for varchar. In this case, the description column (column 3) is the only other text column that we need to do this for.

In [6]:
description_values = get_col_set("boston.csv", 2)
description_values_lengths = [len(value) for value in description_values]
max_description_value_length = max(description_values_lengths)
print(max_description_value_length)

58


## Creating the Table

Now, we will create the boston_crimes table inside the crimes schema of the crime_db database.
First, let's identify suitable column names:

In [7]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


We will stick with these column names for our table.  
Let's remember what kind of data is contained in each column.

In [8]:
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


Column 1: INTEGER  
Column 2: INTEGER  
Column 3: VARCHAR(58)  
Column 4: DATE
Column 5: ENUM (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday)  
Column 6: NUMERIC(10, 8) Max/min value of latitude of longitude is 90/-90  
Column 7: NUMERIC(10, 8)

We must first create the ENUM data type before we can use it in the CREATE TABLE statement.

In [13]:
cursor.execute("""
    CREATE TYPE day_of_week AS ENUM (
        'Monday', 
        'Tuesday', 
        'Wednesday', 
        'Thursday', 
        'Friday', 
        'Saturday', 
        'Sunday'
    );
""")

Now we can create the table.

In [14]:
cursor.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(58),
        date DATE,
        day_of_the_week day_of_week,
        latitude NUMERIC(10, 8),
        longitude NUMERIC(10, 8)
    );
""")

## Loading the Data

We will now load the csv file into our boston_crimes table using copy_expert.

In [17]:
with open("boston.csv", "r") as f:
    cursor.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

Let's verify that data was loaded into the table.

In [20]:
cursor.execute("SELECT * FROM crimes.boston_crimes LIMIT 5;")
cursor.fetchall()

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943')),
 (4,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.33418175'),
  Decimal('-71.07866441')),
 (5,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.27536542'),
  Decimal('-71.09036101'))]